# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

> It uses sliding window view instead of sliding window for splitting the dataset

In [2]:
from dvats.all import *
from tsai.data.preparation import prepare_forecasting_data
from tsai.data.validation import get_long_term_forecasting_splits, get_forecasting_splits
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Get configuration parameters from yml
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [3]:
import utils.config as cfg
config, job_type = cfg.get_artifact_config_embeddings(print_flag = False)

### Show configuration artifact

In [4]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/mvp:latest
input_ar: None
cpu: False


## Build W&B artifact

In [6]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03b_embeddings-sliding_window_view"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03b_embeddings-sliding_window_view


In [7]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [9]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [11]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [12]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb:   1 of 1 files downloaded.  


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [14]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'TitlABP:v0'

### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [15]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'TitlABP:v0'

In [16]:
df = input_ar.to_df()
df.head()

wandb:   1 of 1 files downloaded.  


,time,abp
1970-01-01 00:00:00,0,6832.0
1970-01-01 00:00:01,1,6928.0
1970-01-01 00:00:02,2,6968.0
1970-01-01 00:00:03,3,6992.0
1970-01-01 00:00:04,4,6980.0


In [17]:
df.shape

(40000, 2)

### Split data with Sliding Window

In [20]:
w = enc_run.config['w']
stride = enc_run.config['stride']
enc_input, _ = prepare_forecasting_data(df, fcst_history = w)
enc_input = enc_input[::stride]
enc_input.shape

(2665, 2, 30)

### Split data with Sliding Window View

In [23]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

--> Use CUDA
Use CUDA -->
--> Nested atrr
--> Embs
--> Concat
--> Mean
--> Numpy


In [24]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings-SWV', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [25]:
run.finish()